In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import logistic
from scipy.special import logit

import torch
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset
from torch.cuda.amp import autocast, GradScaler

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Train with GPU support.")
else:
    device = torch.device('cpu')
    print("No GPU found, train with CPU support.")

import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt


# own utils
from utils.graph import *
from utils.tram_models import *
from utils.tram_model_helpers import *
from utils.tram_data import *
from utils.continous import *
from utils.sampling_tram_data import *

# 1. Experiments and Paths

In [ ]:
experiment_name = "dev_multiinput"   ## <--- set experiment name
seed=42
np.random.seed(seed)

LOG_DIR="/home/bule/TramDag/dev_experiment_logs"
EXPERIMENT_DIR = os.path.join(LOG_DIR, experiment_name)
DATA_PATH = EXPERIMENT_DIR # <----------- change to different source if needed
CONF_DICT_PATH = os.path.join(EXPERIMENT_DIR, f"configuration.json")

os.makedirs(EXPERIMENT_DIR,exist_ok=True)
# check if configration dict already exists if not create:

if os.path.exists(CONF_DICT_PATH):
    configuration_dict=load_configuration_dict(CONF_DICT_PATH)
    print(f"Loaded existing configuration from {CONF_DICT_PATH}")
else:
    configuration_dict=create_and_write_new_configuration_dict(experiment_name,CONF_DICT_PATH,EXPERIMENT_DIR,DATA_PATH,LOG_DIR)
    print(f"Created new configuration file at {CONF_DICT_PATH}")

# 2.  Data

In [ ]:


# TODO develop meaningful experiment for interactions



# Fz(z)=Fy(y)
# Fz(h(y|x))=Fy(y)    | z= h(y|x)

# Generate x2

# h(y|x1,x2,x3,x4,x5)= Bernsteinpol(x1,x2) +f2(x2,x3) + f3(x4, x2) + beta * x5        | bernsteinpol is just linearized assumed with a constant factor say 0.42
# h(y|x1,x2,x3,x4,x5)= Bernsteinpol(x1,x2) +f2(x2,x3) + f3(x4, x2) + beta * x5                                          | replace h(y|..) with z
# z                  = 0.42*x2 + beta2 * x1                                           | reformulate to x2
# x2                 = (z-beta2 * x1 )/0.42                                           | sample z from standart logistic via uniform and logit(np.random.uniform(size=n_obs))
# x2                 = (z-beta2 * x1 )/0.42                                           | set beta = 2 (on the edge of the graph)
# x2                 = (z-2 * x1 )/0.42                                               |



### random dgp for testing model pipeline workflow.

In [ ]:
from scipy.special import logit
from mpl_toolkits.mplot3d import Axes3D


# Define the functions used in the DGP
def f1(x1, x2):
    return np.sin(np.pi * x1) * np.cos(np.pi * x2)

def f2(x2, x3):
    return np.exp(-((x2 - 1)**2 + (x3 - 1)**2))

def f3(x4, x2):
    return (x4 * x2) / (1 + x4**2 + x2**2)

def dgp_continuous_interactions(n_obs=10000, seed=42):
    np.random.seed(seed)

    # Independent variables
    x1 = np.random.uniform(0, 2, size=n_obs)
    x2 = np.random.uniform(0, 2, size=n_obs)
    x3 = np.random.uniform(0, 2, size=n_obs)
    x4 = np.random.uniform(0, 2, size=n_obs)
    x5 = np.random.normal(0, 1, size=n_obs)

    # Response variable with interactions
    y = f1(x1, x2) + f2(x2, x3) + f3(x4, x2) + 1.5 * x5

    df = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 'x5': x5, 'x6': y})
    return df

# Generate data
df = dgp_continuous_interactions()

# Visualize the 3 interaction functions
fig = plt.figure(figsize=(18, 5))

# f1(x1, x2)
ax = fig.add_subplot(131, projection='3d')
x = np.linspace(0, 2, 50)
y = np.linspace(0, 2, 50)
X, Y = np.meshgrid(x, y)
Z = f1(X, Y)
ax.plot_surface(X, Y, Z, alpha=0.8)
ax.set_title("f1(x1, x2) = sin(pi*x1)*cos(pi*x2)")
ax.set_xlabel("x1")
ax.set_ylabel("x2")
ax.set_zlabel("f1")

# f2(x2, x3)
ax = fig.add_subplot(132, projection='3d')
X, Y = np.meshgrid(x, y)
Z = f2(X, Y)
ax.plot_surface(X, Y, Z, alpha=0.8)
ax.set_title("f2(x2, x3) = exp(-((x2-1)^2 + (x3-1)^2))")
ax.set_xlabel("x2")
ax.set_ylabel("x3")
ax.set_zlabel("f2")

# f3(x4, x2)
ax = fig.add_subplot(133, projection='3d')
X, Y = np.meshgrid(x, y)
Z = f3(X, Y)
ax.plot_surface(X, Y, Z, alpha=0.8)
ax.set_title("f3(x4, x2) = (x4*x2)/(1+x4^2+x2^2)")
ax.set_xlabel("x4")
ax.set_ylabel("x2")
ax.set_zlabel("f3")

plt.tight_layout()
plt.show()

In [ ]:
def dgp(n_obs=10_000, n_vars=6, seed=None):
    """
    Generate a synthetic dataset with n_vars variables and n_obs observations,
    where each variable contains a constant value: x2 = 2, x3 = 3, ..., x{n_vars+1} = n_vars+1.

    Args:
        n_obs (int): Number of observations (rows).
        n_vars (int): Number of variables (columns).
        seed (int or None): Random seed (not used here since data is deterministic).

    Returns:
        pd.DataFrame: Generated dataset.
    """
    values = np.arange(2, 2 + n_vars)  # [2, 3, ..., n_vars + 1]
    data = np.tile(values, (n_obs, 1))  # Repeat each value across n_obs rows
    columns = [f"x{i+1}" for i in range(n_vars)]
    df = pd.DataFrame(data, columns=columns)
    return df

df =dgp(n_obs=10_000, n_vars=6)

In [ ]:
# 1. Split the data
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# 2. Compute quantiles from training data
quantiles = train_df.quantile([0.05, 0.95])
min_vals = quantiles.loc[0.05]
max_vals = quantiles.loc[0.95]

# 3. Normalize all sets using training quantiles
def normalize_with_quantiles(df, min_vals, max_vals):
    return (df - min_vals) / (max_vals - min_vals)

# train_df = normalize_with_quantiles(train_df, min_vals, max_vals)
# val_df = normalize_with_quantiles(val_df, min_vals, max_vals)
# test_df = normalize_with_quantiles(test_df, min_vals, max_vals)

print(f"Train size: {len(train_df)}, Validation size: {len(val_df)}, Test size: {len(test_df)}")
df.head()

In [ ]:
# os.remove(os.path.join(EXPERIMENT_DIR, "adj_matrix.npy"))

In [ ]:
# --- Editable Parameters ---
variable_names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6']
data_type={'x1':'cont','x2':'cont','x3':'cont','x4':'cont','x5':'cont','x6':'cont'}  # continous , images , ordinal

interactive_adj_matrix(CONF_DICT_PATH ,seed=5)

In [ ]:
x6~ci(x2,x3)+cs(x1,x5)+cs(x4,x5)

In [ ]:
adj_matrix = np.load(os.path.join(EXPERIMENT_DIR, "adj_matrix.npy"),allow_pickle=True)

In [ ]:
adj_matrix

In [ ]:
import numpy as np
import re



In [ ]:
adj_matrix

In [ ]:
data_type

In [ ]:
nn_names_matrix= create_nn_model_names(adj_matrix,data_type)
nn_names_matrix

In [ ]:
conf_dict=get_nodes_dict(adj_matrix, nn_names_matrix, data_type, min_vals, max_vals)

In [ ]:
# conf_dict=get_configuration_dict(adj_matrix,nn_names_matrix, data_type)
# # write min max to conf dict
# for i,node in enumerate(data_type.keys()):
#     conf_dict[node]['min']=min_vals[i].tolist()
#     conf_dict[node]['max']=max_vals[i].tolist()
# conf_dict

ordered paretns check

In [ ]:
ordered_parents_datatype, ordered_transformation_terms_in_h, ordered_transformation_term_nn_models_in_h=ordered_parents(node, conf_dict)

In [ ]:
ordered_parents_datatype
ordered_transformation_terms_in_h

In [ ]:
ordered_transformation_term_nn_models_in_h

In [ ]:
train_loader, val_loader = get_dataloader("x6", conf_dict, train_df, val_df, batch_size=10, verbose=True)


In [ ]:
next(iter(train_loader))